In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [2]:
#Load the trained model, scaler pickle, onehot
model = load_model('model.h5')

#load encoder and scaler
with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)


with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
                            


In [3]:
#Example Input Data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [5]:
#OneHot encode Geography
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\nishk\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0

In [16]:
input_df1 = pd.DataFrame([input_data])
input_df1


CreditScore Geography Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France   Male   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000

In [17]:
#Encode categorical variables
input_df1['Gender'] = label_encoder_gender.transform(input_df1['Gender'])
input_df1

CreditScore Geography  Gender  Age  Tenure  Balance  NumOfProducts  \
0          600    France       1   40       3    60000              2   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1            50000

In [18]:
input_df1 = pd.concat([input_df1.drop("Geography", axis=1), geo_encoded_df], axis=1)
input_df1

CreditScore  Gender  Age  Tenure  Balance  NumOfProducts  HasCrCard  \
0          600       1   40       3    60000              2          1   

   IsActiveMember  EstimatedSalary  Geography_France  Geography_Germany  \
0               1            50000               1.0                0.0   

   Geography_Spain  
0              0.0

In [19]:
#Scaling the input data
input_scaled = scaler.transform(input_df1)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [20]:
#making prediction
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 240ms/step


array([[0.0476321]], dtype=float32)

In [21]:
prediction_probability = prediction[0][0]
prediction_probability

0.0476321

In [22]:
if prediction_probability > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
